In [1]:
import torch
import numpy
print(torch.__version__)

1.13.1


In [2]:
print(torch.version.cuda)

11.6


In [3]:
cuda = torch.device('cuda')

In [17]:
import csv
import sys

csv.field_size_limit(int(sys.maxsize / 100000000000))
texts = []
signs = []
with open('./signed.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        texts.append(row[2])
        signs.append(row[5])
word_counts = []
for text in texts:
    words = text.split()
    word_counts.append(len(words))
print(sum(word_counts) / len(word_counts))

280.2806927606512


In [5]:
import numpy as np
from gensim.models import Word2Vec
import time

sentences = []
for text in texts:
    sentences.append(text.split(' '))
start = time.time()
model = Word2Vec(sentences=sentences, vector_size=50, window=5, min_count=1, workers=20)
model.save("word2vec.model")
end = time.time()
print(end - start)

C:\ProgramData\anaconda3-64\envs\new_pytorch_env\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


56.463613748550415


In [6]:
print(model.wv.most_similar('газ'))

[('топлив', 0.82228022813797), ('нефт', 0.8111058473587036), ('энергоносител', 0.7799270749092102), ('электроэнерг', 0.7759563326835632), ('энергоресурс', 0.769345223903656), ('спг', 0.747270405292511), ('нефтепродукт', 0.7465770244598389), ('газов', 0.7330261468887329), ('газа»', 0.7305629253387451), ('нефтепровод', 0.7055534720420837)]


In [7]:
vectors = []
for text in texts[1:]:
    vectors.append([model.wv[w] for w in text.split(' ')])

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

count = int(len(vectors) * 0.7)
X = vectors
classes = [-1, 0, 1]
signs = signs[1:]
for i in range(len(signs)):
    signs[i] = int(signs[i])
y = torch.zeros(len(vectors), 3, device='cuda:0')
for i in range(len(vectors)):
    y[i][classes.index(signs[i])] = 1

In [19]:
Xtrain, Xtest = X[:count], X[count:]
Ytrain, Ytest = y[:count], y[count:]

In [ ]:
lstm = nn.LSTM(input_size=50, hidden_size=3, num_layers=1)
lstm.to("cuda:0")
optimizer = optim.Adam(lstm.parameters())
loss_fn = nn.MSELoss()

lstm.train()
for epoch in range(500):
    total_loss = 0
    global_start = time.time()
    for i in range(len(Xtrain)):
        hidden = (
            torch.rand(1, 3, device='cuda:0'),
            torch.rand(1, 3, device='cuda:0')
        )
        x = Xtrain[i]
        x = np.array(x)
        _, hidden = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
        yPred = hidden[0]
        loss = loss_fn(yPred, Ytrain[i].reshape(1, 3))
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    count = 0
    for i in range(len(Xtrain)):
        hidden = (
            torch.rand(1, 3, device='cuda:0'),
            torch.rand(1, 3, device='cuda:0')
        )
        x = Xtrain[i]
        x = np.array(x)
        _, hidden = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
        yPred = hidden[0]
        yPred = yPred.reshape(-1)
        _max = 0
        j_max = 0
        for j in range(len(yPred)):
            if _max < yPred[j]:
                _max = yPred[j]
                j_max = j
        if Ytrain[i][j_max] == 1:
            count += 1
    total_loss /= len(Xtrain)
    global_end = time.time()
    print(f"Epoch {epoch}: Time: {global_end - global_start}s: Loss: {total_loss} Accuracy: {count / len(X)}")

Epoch 0: Time: 225.90056800842285s: Loss: 0.1659883199509949 Accuracy: 0.48246957027759113
Epoch 1: Time: 227.76250529289246s: Loss: 0.09308606472966338 Accuracy: 0.5514551538285682
Epoch 2: Time: 229.1815469264984s: Loss: 0.08169754562429506 Accuracy: 0.540391689585657
Epoch 3: Time: 229.1042821407318s: Loss: 0.07983407995044933 Accuracy: 0.5599553859049942
Epoch 4: Time: 229.00746297836304s: Loss: 0.07950238991123033 Accuracy: 0.5658161816153905
